<a href="https://colab.research.google.com/github/palemao/bitcoin-price-prediction/blob/main/BTC_tweet_senitment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
!pip install -U keras-tuner
import kerastuner as kt
import IPython
!pip install pyyaml h5py 
from keras import regularizers 

     |████████████████████████████████| 71kB 5.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=83bbf14c6681192f766278c80f112238c194d60b957ca424cb5ed4dae2a80318
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=95cb8097b301110ee3412159b7584dd4833f903438517106213fc8c51668c5bf
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
df= pd.read_csv('/uniquetweets.csv') 
df.head()
print(len(df))

18187


### Definitions

Vocab_size = The possible number of words 

embedding_dim = The vector size for each word which represents how similar or different words are from each other

max_length = Maximum Tweet Length

padding_type = Tweets shorter than 100 words are padded with 0 after the last word to ensure all tensors are the same length before being fed into neural network

oov_tok = placeholder for words that could not be encoded

Training index = Creates an index number 80% of the length of the entire data set, this number is used to seperate the dataset into train and test sets.

In [3]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_index = int(len(df)*0.8)

Creating test and train datasets:

In [4]:
labels = np.asarray(df['sent_score'])

test_tweets = np.asarray(df[training_index:]['Tweet'].tolist())
test_labels = np.asarray(labels[training_index:])

tweets = np.asarray(df[:training_index]['Tweet'].tolist())
labels = np.asarray(labels[:training_index])


print("Original Labels \n", labels[25:40])



Original Labels 
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0.]


Takes words from tweets and encodes them into numbers, placing "<OOV>" for unknown words:

Each word has a unqiue number now, print word_index to see.

In [55]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(tweets)
word_index= tokenizer.word_index

1. Encoded/tokenised words can be used to make "token sentences" from tweets.

2. `padding_sequences()` adds 0's after tweets that are less than 100 words and truncates tweets more than 100 words. Therefore each encoded tweet is the same length of 100



In [6]:
#training padded
sequences = tokenizer.texts_to_sequences(tweets)
padded = np.array(pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type))
print(tweets[123])
print(padded[123])

  Ronald_vanLoon " Ronald_vanLoon: Raiden Network #IoT Demo 
by:brainbot_tech |

#InternetofThings #AI #Artificialintel…
[5885 5886 5885 5886 5887  203  425 1543   43 8767  354 1544  414 8768
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [7]:
#testing padded
testing_seq = tokenizer.texts_to_sequences(test_tweets)
testing_padded = np.asarray(pad_sequences(testing_seq, maxlen=max_length ,padding=padding_type, truncating=trunc_type))
print(test_tweets[123])
print(testing_padded[123])

Puerto Rico creates a working group to review blockchain projects - 
[3711 4398 1955    6  692  379    4  980   10  766    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


Define BinaryCrossentropy() loss function used for binary classification tasks. 

Define early stopping callback to prevent over fitting of model and to save time when using trial and error to create models. This callback stops training if there is an insignificant reduction/increase in validation loss, there is a patience of 2, to provide some leeway 

In [9]:
tf.keras.losses.BinaryCrossentropy(
    from_logits=False, label_smoothing=0, reduction="auto", name="binary_crossentropy"
)

callback  = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=2,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

Define Hyper-Model, here using Gaussian Statistics we can use **baysian optimisation** method, where combination of hyperparamters are tested based on Gaussian Distribution which highlights where the next best combination is likley to be based on previous trials. This method has shown greater results compared to other hyperparameter optimastion methods such as grid search or hyperband.

In [31]:
def model_builder(hp):  #Things to try: 1. drop out choice  2.Number of layers 
  model = keras.Sequential()
  
  ##first layer
  hp_units1 = hp.Int('units1', min_value = 32, max_value = 512, step = 32)
  dp1 = hp.Choice('dp1', values = [0.25,0.5,0.75]) 

  #second layer
  hp_units2 = hp.Int('units2', min_value = 32, max_value = 512, step = 32)
  dp2 = hp.Choice('dp2', values = [0.25,0.5,0.75])
 
  #third layer
  hp_units3 = hp.Int('units3', min_value = 32, max_value = 512, step = 32)
  dp3 = hp.Choice('dp3', values = [0.25,0.5,0.75])
 
  model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True)) #embedding layer
 
  model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(hp_units1,  return_sequences=True))) #layer 1
  tf.keras.layers.Dropout(dp1),
 
  model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(hp_units2))) # layer2
  tf.keras.layers.Dropout(dp2),
 
  model.add(keras.layers.Dense(units=hp_units3, activation='relu')) #layer 3
  tf.keras.layers.Dropout(dp3),

 
  model.add(keras.layers.Dense(1)) #output  
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = 1e-4),
                loss = tf.keras.losses.BinaryCrossentropy(),
                metrics = ['accuracy'])
  
  return model 

In [36]:
tuner = kt.BayesianOptimization(model_builder,  
                     objective='val_accuracy',
                     max_trials = 2,
                     directory = 'my_dir',
                     project_name = 'into_to_kt',
                     overwrite=True,
                     )

In [37]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [42]:
tuner.search(padded, labels, epochs = 1, validation_data = (testing_padded, test_labels), batch_size=32, callbacks = [ClearTrainingOutput()])

INFO:tensorflow:Oracle triggered exit


In [43]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

for i in range(1,4):
   print(best_hps.get(f"units{i}"))
   print(best_hps.get(f"dp{i}"))
   
#tuner.results_summary()

512
0.5
128
0.25
448
0.25


Feed the best hyperparameter combination into a final neural network for training 20 epochs and yield predictions

In [46]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(best_hps.get("units1"), return_sequences=True)),
    tf.keras.layers.Dropout(best_hps.get("dp1")),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(best_hps.get("units2"))),
    tf.keras.layers.Dropout(best_hps.get("dp2")),
    tf.keras.layers.Dense(best_hps.get("units3"), activation='relu'),
    tf.keras.layers.Dropout(best_hps.get("dp3")),
    tf.keras.layers.Dense(1)
])

opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),optimizer=opt ,metrics=["accuracy"])

This is the best hyperparameter model
```
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True,
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512,return_sequences=True)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),optimizer=opt ,metrics=["accuracy"])
```

Train the model with `batch_size=32`


In [30]:
num_epochs = 20
history = model.fit(padded, labels, epochs=num_epochs, validation_data=(testing_padded, test_labels), batch_size=32 ,verbose=1, callbacks =[callback])

Epoch 1/10
455/455 [==============================] - 25s 41ms/step - loss: 0.0876 - accuracy: 0.9938 - val_loss: 0.6821 - val_accuracy: 0.9382
Epoch 2/10
455/455 [==============================] - 14s 32ms/step - loss: 0.0868 - accuracy: 0.9941 - val_loss: 0.6869 - val_accuracy: 0.9390
Epoch 3/10
455/455 [==============================] - 15s 34ms/step - loss: 0.0868 - accuracy: 0.9942 - val_loss: 0.6961 - val_accuracy: 0.9365
Epoch 4/10
455/455 [==============================] - 15s 33ms/step - loss: 0.0867 - accuracy: 0.9942 - val_loss: 0.7127 - val_accuracy: 0.9365
Epoch 5/10
455/455 [==============================] - 15s 32ms/step - loss: 0.0882 - accuracy: 0.9939 - val_loss: 0.7098 - val_accuracy: 0.9379
Epoch 6/10
455/455 [==============================] - 15s 34ms/step - loss: 0.0877 - accuracy: 0.9942 - val_loss: 0.7196 - val_accuracy: 0.9398
Epoch 7/10
455/455 [==============================] - 15s 32ms/step - loss: 0.0874 - accuracy: 0.9943 - val_loss: 0.7221 - val_accuracy:

In [ ]:
#model.save('/_Pneu3.h5') 

Load Most Accurate Model after many "tuning sessions" with keras tuner:

In [47]:
model = keras.models.load_model('/_Pneu3.h5')

In [48]:
model.evaluate(x=testing_padded, y=test_labels, batch_size=10)

364/364 [==============================] - 7s 8ms/step - loss: 0.6535 - accuracy: 0.9412


[0.6535272002220154, 0.9411764740943909]